---
title: Where to live
format: 
  html:
    grid:
        body-width: 1024px
    # page-layout: full
execute:
  echo: false
---

In [16]:
#import library
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

#render ploty
pio.renderers.default = "notebook"

#import data
us_states=gpd.read_file("../final_data/tl_2020_us_state.shp")
df = pd.read_csv("../final_data/final_data.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

#normalize data
columns_to_normalize = ['personal.income', 'Median.Rent', 'median.sale.price',
                        'Walkable', 'Unemployment', 'Purchase_power_100_dollars',
                        'Violent_Crime_Rate_per_100k', 'Non_Violent_Crime_Rate_per_100k']

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [17]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Choosing Where to Live",style={"color": "white"}),
    html.Div([
        html.Label("Importance of Rent",style={"color": "white"}),
        dcc.Slider(
            id="Median Rent",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Income",style={"color": "white"}),
        dcc.Slider(
            id="Personal Income",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Home Sale Price",style={"color": "white"}),
        dcc.Slider(
            id="Median Sale Price",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Walkablity",style={"color": "white"}),
        dcc.Slider(
            id="Walkable",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Job Security and Opportunity",style={"color": "white"}),
        dcc.Slider(
            id="Unemployment",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Purchasing Power",style={"color": "white"}),
        dcc.Slider(
            id="Purchasing Power",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Violent Crime Rate",style={"color": "white"}),
        dcc.Slider(
            id="Crime Rate",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        # Repeat this pattern for other variables
    ], style={"width": "50%", "margin": "auto"}),
    dcc.Graph(id="city-graph")
])

# Define callback to update the graph based on user input
@app.callback(
    Output("city-graph", "figure"),
    [Input("Median Rent", "value"),
     Input("Personal Income", "value"),
     Input("Median Sale Price","value"),
     Input("Walkable","value"),
     Input("Unemployment","value"),
     Input("Purchasing Power","value"),
     Input("Crime Rate","value")])
    # Add more Inputs for other variables
def update_graph(importance_personal_income, importance_median_rent,importance_median_sale_price,importance_walkable,importance_unemployment,importance_purchasing_power,importance_crime):
    # Weighted sum of variables
    df["Weighted_Sum"] = (df["Median.Rent"] * importance_personal_income +
                          df["personal.income"] * importance_median_rent + df["median.sale.price"]*importance_median_sale_price +df["Walkable"]*importance_walkable+df["Unemployment"]*importance_unemployment+
                          df["Purchase_power_100_dollars"]*importance_purchasing_power+df["Violent_Crime_Rate_per_100k"]*importance_crime)
    
    # Define color scale mapping each political affiliation to a specific color
    color_scale = {
        "D": "blue",
        "R": "red",
        "I": "yellow",
        "Nonpartisan": "grey"
    }
    
    # Plotly scatter plot
    fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', size="Weighted_Sum", color="Political_Affiliation",
                         color_discrete_map=color_scale, custom_data=["REGION", "Weighted_Sum"])
    
    fig.update_layout(title="City Data Visualization on US Map",
                      mapbox_style="carto-positron",
                      mapbox_zoom=3,
                      mapbox_center={"lat": 37.0902, "lon": -95.7129})
    
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>Weighted Sum: %{customdata[1]:.2f}')
    
    return fig
# Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=8052)

In [18]:
df = pd.read_csv("../final_data/final_data.csv")


In [24]:
#color scale
color_scale = {
    "D": "blue",
    "R": "red",
    "I": "yellow",
    "Nonpartisan": "grey"
    }
#colors
color_scale = {
    "D": "blue",
    "R": "red",
    "I": "yellow",
    "Nonpartisan": "grey"
    }

#def east/west
def east_west(region):
    west_coast = ['Seattle', 'San Francisco', 'Los Angeles', 'San Diego', 'Portland']
    east_coast = ['Boston', 'New York', 'Washington, DC', 'Miami', 'Philadelphia', 'Atlanta', 'Baltimore', 'Virginia Beach', 'Charlotte', 'Tampa', 'Orlando', 'Miami']
    if region in west_coast:
        return "west"
    elif region in east_coast:
        return 'east'
    else:
        return 'cent'
    
df["coast"] = df["REGION"].apply(east_west)

# Sort the DataFrame by 'Year_Fahrenheit' column
# df_sorted = df.sort_values(by='Year_Fahrenheit')


#make df for each part of the country
#east
df_east = df[df['coast'] == 'east']
df_east = df_east.sort_values(by='Year_Fahrenheit')
#west
df_west = df[df['coast'] == 'west']
df_west = df_west.sort_values(by='Year_Fahrenheit')
#cent
df_cent = df[df['coast'] == 'cent']
df_cent = df_cent.sort_values(by='Year_Fahrenheit')

# For average temperature in west region
fig1 = px.bar(df_west, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Western Metro Areas')

# For average temperature in central region
fig2 = px.bar(df_cent, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Central Metro Areas')
# For average temperature in east region
fig3 = px.bar(df_east, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Eastern Metro Areas')

# Create subplots with 1 row and 3 columns
fig = make_subplots(rows=1, cols=3, subplot_titles=('West Coast Metro Areas', 'Central Metro Areas', 'East Coast Metro Areas'))

# Add the plots to the subplots
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=1, col=3)

# Update layout
fig.update_layout(title_text='Average Temperature by Metro Area', yaxis_title='Temperature (F)', showlegend=False)
# Update bar width
fig.update_traces(width=.75)
# Show the plot
fig.show()

In [20]:
#walkability

#sort
df = df.sort_values(by='Walkable', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Walkable', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Walkability Score', title='Walkability Score by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [21]:
#walkability
#sort
df = df.sort_values(by='Purchase_power_100_dollars', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Purchase_power_100_dollars', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Purchasing Power by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [22]:
#median rent
#sort
df = df.sort_values(by='Median.Rent', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Median.Rent', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Rent by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [23]:
#median.sale.price
#sort
df = df.sort_values(by='median.sale.price', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='median.sale.price', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Home Sale Price by Metro Area')

#Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()